In [1]:
import torch
from torch import nn

## understand the lstm parameters

In [2]:
#LSTM
net = nn.LSTM(100, 128) #Assume only one layer
print(net.weight_ih_l0.shape)# (4*hidden_size, input_size)
print(net.weight_hh_l0.shape)#(4*hidden_size, hidden_size)
print(net.bias_ih_l0.shape)#(4*hidden_size)
print(net.bias_hh_l0.shape)#(4*hidden_size)
w_ii, w_if, w_ic, w_io = net.weight_ih_l0.chunk(4, 0) #每个都是(hidden_size,input_size)
print(w_ii.shape)
w_hi, w_hf, w_hc, w_ho = net.weight_hh_l0.chunk(4, 0) #每个都是(hidden_size,hidden_size)
print(w_hi.shape)
b_ii, b_if, b_ic, b_io = net.bias_ih_l0.chunk(4, 0) #每个都是(hidden_size)
print(b_ii.shape)
b_hi, b_hf, b_hc, b_ho = net.bias_hh_l0.chunk(4, 0) #每个都是(hidden_size)
print(b_hi.shape)

torch.Size([512, 100])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([128, 100])
torch.Size([128, 128])
torch.Size([128])
torch.Size([128])


## 可以对不同的gate进行自定义的初始化
但一般建议:
* 将gates的权重正交初始化， 
* forget gate的bias初始值(LSTM时)取1.0

另外注意：W chunk之后的各个部分其实还在原来的内存区域，所以**对各个部分的初始化会直接反应到整个W中的相应区域**,不需要去反过来赋值或重组

In [3]:
#默认均匀随机初始化
w_if

tensor([[-0.0626,  0.0224,  0.0790,  ..., -0.0398,  0.0043, -0.0535],
        [-0.0717, -0.0532,  0.0171,  ..., -0.0884, -0.0362, -0.0370],
        [-0.0738,  0.0284, -0.0317,  ...,  0.0549,  0.0575, -0.0075],
        ...,
        [ 0.0065,  0.0557, -0.0182,  ...,  0.0415,  0.0431, -0.0754],
        [-0.0809, -0.0491,  0.0783,  ...,  0.0447, -0.0278, -0.0652],
        [-0.0777, -0.0701, -0.0436,  ..., -0.0006,  0.0531, -0.0725]],
       grad_fn=<SplitBackward>)

In [4]:
#nn.init.orthogonal_(self.rnn.weight_ih_l0) #可以直接对所有的gates正交初始化
nn.init.orthogonal_(w_if)#orthogonal initialization

tensor([[-0.0541,  0.0111,  0.1698,  ..., -0.0180, -0.2143, -0.1415],
        [-0.0221,  0.0007,  0.0994,  ..., -0.0352,  0.1932, -0.1889],
        [ 0.0181,  0.0853,  0.0252,  ...,  0.0795, -0.0963, -0.0718],
        ...,
        [-0.1255,  0.1350, -0.0790,  ..., -0.0284, -0.0436, -0.0458],
        [ 0.0839,  0.1167, -0.0585,  ...,  0.0896, -0.0150, -0.0669],
        [-0.0345, -0.1474, -0.1191,  ..., -0.0168,  0.0529, -0.0357]],
       grad_fn=<SplitBackward>)

In [5]:
w_ii, w_if_new, w_ic, w_io = net.weight_ih_l0.chunk(4, 0)
print(w_if_new)#自动跟着改变，因为内存相同

tensor([[-0.0541,  0.0111,  0.1698,  ..., -0.0180, -0.2143, -0.1415],
        [-0.0221,  0.0007,  0.0994,  ..., -0.0352,  0.1932, -0.1889],
        [ 0.0181,  0.0853,  0.0252,  ...,  0.0795, -0.0963, -0.0718],
        ...,
        [-0.1255,  0.1350, -0.0790,  ..., -0.0284, -0.0436, -0.0458],
        [ 0.0839,  0.1167, -0.0585,  ...,  0.0896, -0.0150, -0.0669],
        [-0.0345, -0.1474, -0.1191,  ..., -0.0168,  0.0529, -0.0357]],
       grad_fn=<SplitBackward>)


In [6]:
#观察forget gate bias的默认初始化
print(b_if)
print(b_hf)

tensor([-4.8035e-02, -6.2069e-02,  1.4225e-02,  5.8883e-02, -8.4355e-02,
        -1.8253e-02, -7.4605e-02, -4.4818e-02,  5.3677e-02,  6.9491e-02,
        -4.7941e-02,  7.7492e-02, -5.7955e-02, -4.4675e-02, -3.8080e-02,
         1.4237e-02, -1.0186e-02,  3.6231e-02, -2.5299e-02, -4.5784e-02,
         5.6310e-02,  7.6527e-02, -1.2838e-02, -6.4982e-02,  2.9514e-02,
         1.6034e-02, -3.9057e-02, -4.2777e-02, -3.9535e-02,  3.3162e-03,
         2.7241e-02,  8.4419e-03,  6.4859e-02,  2.7500e-02, -6.5306e-02,
        -5.3536e-02, -1.3858e-05,  4.7566e-02,  2.8924e-03,  5.5258e-02,
         2.5063e-02,  5.8828e-02,  7.7831e-02, -2.4024e-02,  4.7317e-02,
         9.2443e-03,  1.6396e-02,  3.6019e-02,  2.2863e-02,  4.2083e-02,
         1.5684e-02, -8.6889e-02, -3.1551e-02, -7.0145e-03,  3.7256e-02,
        -6.9480e-02,  4.0860e-02, -3.5777e-03, -1.3850e-02,  6.3861e-02,
        -8.0556e-02,  4.5408e-02,  3.3153e-02, -4.4095e-02,  3.2675e-03,
        -6.7233e-03,  2.4190e-02,  5.4704e-02, -5.0

In [7]:
#forget gate的bias初始值(LSTM时)取1.0
nn.init.ones_(b_if) #也可以采用赋值的方式
nn.init.ones_(b_hf)
print(b_if)
print(b_hf)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], grad_fn=<SplitBackward>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.

In [8]:
b_ii, b_if_new, b_ic, b_io = net.bias_ih_l0.chunk(4, 0) #每个都是(hidden_size)
print(b_if_new) #自动改变，因为内存相同
b_hi, b_hf_new, b_hc, b_ho = net.bias_hh_l0.chunk(4, 0) #每个都是(hidden_size)
print(b_hf_new)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], grad_fn=<SplitBackward>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.

In [9]:
# #对各个gate分别初始化之后，重组:重组当然可以，但是没必要，对各个gate in_place的初始化会直接反应到net.weight_ih_l0
# print(net.weight_ih_l0)
# #.cat 和 .stack的区别在于 cat会增加现有维度的值,可以理解为拼接，stack会新加增加一个维度，可以理解为叠加
# net.weight_ih_l0 = nn.Parameter(torch.cat((w_ii, w_if, w_ic, w_io), dim=0),
#                                 requires_grad=True)
# print(net.weight_ih_l0)
# w_ii, w_if, w_ic, w_io = net.weight_ih_l0.chunk(4, 0)
# print(w_if)

In [10]:
#GRU:不再赘述
net = nn.GRU(100,128)
w_ir, w_ii, w_in = net.weight_ih_l0.chunk(3, 0)
w_hr, w_hi, w_hn = net.weight_hh_l0.chunk(3, 0)

In [11]:
#看一个复杂点的例子：多层的会有层号，双向模型逆向有_reverse后缀
net = nn.LSTM(10, 8, num_layers=2, bidirectional=True)#双层双向
for name, params in net.named_parameters():
    print(name, params.shape)

weight_ih_l0 torch.Size([32, 10])
weight_hh_l0 torch.Size([32, 8])
bias_ih_l0 torch.Size([32])
bias_hh_l0 torch.Size([32])
weight_ih_l0_reverse torch.Size([32, 10])
weight_hh_l0_reverse torch.Size([32, 8])
bias_ih_l0_reverse torch.Size([32])
bias_hh_l0_reverse torch.Size([32])
weight_ih_l1 torch.Size([32, 16])
weight_hh_l1 torch.Size([32, 8])
bias_ih_l1 torch.Size([32])
bias_hh_l1 torch.Size([32])
weight_ih_l1_reverse torch.Size([32, 16])
weight_hh_l1_reverse torch.Size([32, 8])
bias_ih_l1_reverse torch.Size([32])
bias_hh_l1_reverse torch.Size([32])
